<a href="https://colab.research.google.com/github/gagan3012/keytotext/blob/master/Notebooks/Datapipelinek2s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install unidecode

from xml.dom import minidom
from pathlib import Path
import re
import unidecode
import os
import pandas as pd
import glob


In [14]:
def clean_node(node):
    node = node.strip()
    node = node.replace('(', '')
    node = node.replace('\"', '')
    node = node.replace(')', '')
    node = node.replace(',', ' ')
    node = node.replace('_', ' ')
    node = unidecode.unidecode(node)
    return node

In [15]:
def clean_edge(edge):
    edge = edge.replace('(', '')
    edge = edge.replace(')', '')
    edge = edge.strip()
    edge = edge.split()
    edge = "_".join(edge)
    return edge

In [16]:
def camel_case_split(text):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', text)
    d = [m.group(0) for m in matches]
    new_d = []
    for token in d:
        token = token.replace('(', '')
        token_split = token.split('_')
        for t in token_split:
            new_d.append(t.lower())
    return ' '.join(new_d)

In [17]:
def process_triples(mtriples):
    nodes = []

    for triple in mtriples:
        triple_text = triple.firstChild.nodeValue
        individual_triples = triple_text.strip().split(' | ')
        h = clean_node(individual_triples[0])
        r = camel_case_split(clean_edge(individual_triples[1]))
        t = clean_node(individual_triples[2])
        
        nodes.append('<H>')
        nodes.extend(h.split())

        nodes.append('<R>')
        nodes.extend(r.split())

        nodes.append('<T>')
        nodes.extend(t.split())

    return nodes

In [18]:
def get_data(file):
    return_data = []
    xmldoc = minidom.parse(file)
    entries = xmldoc.getElementsByTagName('entry')
    for e in entries:
        mtriples = e.getElementsByTagName('mtriple')
        nodes = ' '.join(process_triples(mtriples))
        lexs = e.getElementsByTagName('lex')
        for l in lexs:
            l = l.firstChild.nodeValue.strip()
            l = unidecode.unidecode(l)
            return_data.append((nodes,l))
    return return_data

In [22]:
def get_file(files):
  train_dict={"input_text":[], "target_text":[]}

  for filename in files_train:
    filename = str(filename)
    data = get_data(filename)
    for data_point in data:
        input_text = data_point[0]
        target_text = data_point[1]
        train_dict['input_text'].append(input_text)
        train_dict['target_text'].append(target_text)
  return pd.DataFrame(train_dict)

In [10]:
!git clone https://gitlab.com/shimorina/webnlg-dataset.git

Cloning into 'webnlg-dataset'...
remote: Enumerating objects: 5106, done.
remote: Total 5106 (delta 0), reused 0 (delta 0), pack-reused 5106
Receiving objects: 100% (5106/5106), 26.08 MiB | 18.37 MiB/s, done.
Resolving deltas: 100% (4008/4008), done.
Checking out files: 100% (1425/1425), done.


In [20]:
files_train = glob.glob("/content/webnlg-dataset/release_v3.0/en/train/**/*.xml", recursive=True)
files_test = glob.glob("/content/webnlg-dataset/release_v3.0/en/test/**/*.xml", recursive=True)
files_dev = glob.glob("/content/webnlg-dataset/release_v3.0/en/dev/**/*.xml", recursive=True)

In [23]:
train = get_file(files_train)
dev = get_file(files_dev)
test = get_file(files_test)

In [26]:
train.to_csv('./webnlg_train.csv', index=False)
dev.to_csv('./webnlg_dev.csv', index=False)
test.to_csv('./webnlg_test.csv', index=False)

In [27]:
!git clone https://github.com/Yale-LILY/dart.git

Cloning into 'dart'...
remote: Enumerating objects: 11376, done.
remote: Counting objects: 100% (11376/11376), done.
remote: Compressing objects: 100% (8918/8918), done.
remote: Total 11613 (delta 2490), reused 11319 (delta 2437), pack-reused 237
Receiving objects: 100% (11613/11613), 267.25 MiB | 35.96 MiB/s, done.
Resolving deltas: 100% (2526/2526), done.
Checking out files: 100% (1520/1520), done.


In [28]:
files_train = glob.glob("/content/dart/data/v1.1.1/dart-v1.1.1-full-train.xml", recursive=True)
files_test = glob.glob("/content/dart/data/v1.1.1/dart-v1.1.1-full-test.xml", recursive=True)
files_dev = glob.glob("/content/dart/data/v1.1.1/dart-v1.1.1-full-dev.xml", recursive=True)

In [29]:
train = get_file(files_train)
dev = get_file(files_dev)
test = get_file(files_test)

In [30]:
train.to_csv('./dart_train.csv', index=False)
dev.to_csv('./dart_dev.csv', index=False)
test.to_csv('./dart_test.csv', index=False)